In [ ]:
# Cell used to give direct access to *this* notebook's local directory
from google.colab import drive
import os

drive.mount('/content/drive/', force_remount=True)
os.chdir("/content/drive/MyDrive/Datathon 2022")

Mounted at /content/drive/


In [ ]:
drive.flush_and_unmount()

# Speech to text - Hugging Face

The Purpose of this Notebook is to develop a feature that is able to clasify text into one of several emotions 

In [ ]:
# Audio to Text Transformation
!pip install -q transformers
!pip install -q datasets

     |████████████████████████████████| 5.5 MB 4.4 MB/s 
     |████████████████████████████████| 7.6 MB 39.2 MB/s 
     |████████████████████████████████| 163 kB 67.2 MB/s 
     |████████████████████████████████| 441 kB 5.0 MB/s 
     |████████████████████████████████| 95 kB 3.9 MB/s 
     |████████████████████████████████| 115 kB 47.1 MB/s 
     |████████████████████████████████| 212 kB 49.5 MB/s 
     |████████████████████████████████| 127 kB 50.4 MB/s 
     |████████████████████████████████| 115 kB 56.0 MB/s 


In [ ]:
# Audio recording
!sudo apt install ffmpeg
!pip install torchaudio ipywebrtc notebook
!jupyter nbextension enable --py widgetsnbextension

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 260 kB 5.4 MB/s 
     |████████████████████████████████| 1.6 MB 34.8 MB/s 
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset, Audio
import torch
 
# load model and tokenizer
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
    
# load dummy dataset and read soundfiles
ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")

# tokenize
input_values = processor(ds[0]["audio"]["array"], return_tensors="pt", padding="longest").input_values  # Batch size 1

# retrieve logits
logits = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

...

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset librispeech_asr_dummy downloaded and prepared to /root/.cache/huggingface/datasets/patrickvonplaten___librispeech_asr_dummy/clean/2.1.0/f2c70a4d03ab4410954901bde48c54b85ca1b7f9bf7d616e7e2a72b5ee6ddbfc. Subsequent calls will reuse this data.


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [ ]:
from ipywebrtc import AudioRecorder, CameraStream
import torchaudio
from IPython.display import Audio

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder


AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [ ]:
recorder.save('Audio.mp4')

In [ ]:
import librosa

speech, rate = librosa.load("./Audio.mp4",sr=16000)
input_audio = processor(speech, return_tensors = 'pt').input_values

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [ ]:
logits = model(input_audio).logits
predicted_ids = torch.argmax(logits, dim =-1)
transcriptions = processor.decode(predicted_ids[0]).lower()
print(transcriptions)

sorry it was an accident i am ok


# Sentiment Analysis - Cohere

The Purpose of this Notebook is to develop a feature that is able to clasify text into one of several emotions (positive and negatives ones) using 

In [ ]:
# Let's install the dependencies
!pip install cohere

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for cohere: filename=cohere-2.8.0-cp37-cp37m-linux_x86_64.whl size=10956 sha256=d6d68409ac6ccd124c508c69c159f848464802ee80678e3bdedba57e33bf28b6
  Stored in directory: /root/.cache/pip/wheels/85/bd/8e/65048fd92114861be1984d02d48bf68b8f2475185178600024
Successfully built cohere


In [ ]:
# Imports of the document
import cohere
from cohere.classify import Example

co = cohere.Client('qhHPck58e98i1UFzeNHJigqcU3KhDOdokly1dN3G') # -> Main instance of coherence

In [ ]:
voice_samples = [
    Example("I need an ambulance", "Emergency"),
    Example("I need an help", "Emergency"),
    Example("Please hurry, I am in pain", "Emergency"),
    Example("I think I’ve broken a bone", "Emergency"),
    Example("I’m bleeding, need some help", "Emergency"),
    Example("Someone hit me and I think I hurt my nose", "Emergency"),
    Example("Call for an ambulance", "Emergency"),
    Example("I can't breathe", "Emergency"),
    Example("I falling son", "Emergency"),
    Example("I’m having trouble breathing", "Emergency"),
    Example("I am in pain", "Emergency"),
    Example("I think I’m having a heart attack", "Emergency"),
    Example("I'm getting very dizzy", "Emergency"),
    Example("I fell and my body hurts a lot", "Emergency"),
    Example("Can anyone help me?", "Emergency"),
    Example("My stomach hurts too much", "Emergency"),
    Example("My chest hurts", "Emergency"),
    Example("My head hurts", "Emergency"),
    Example("My leg hurts", "Emergency"),
    Example("I can’t see anything, I am afraid", "Emergency"),
    Example("My eyes are stinging for a few hours", "Emergency"),
    Example("My tooth has come out", "Emergency"),
    Example("I’ve dislocated my shoulder", "Emergency"),
    Example("I’ve put my back out", "Emergency"),
    Example("Please come quick, I am in such a pain", "Emergency"),
    Example("I fell off a ladder and I am bleeding", "Emergency"),
    Example("My heart is in such pain...", "Emergency"),
    Example("I am having a heart attack", "Emergency"),
    Example("I passed out and have been unconscious", "Emergency"),
    Example("Can you bring an ambulance?", "Emergency"),
    Example("Help Help I need medical assistance", "Emergency"),
    Example("Please hurry! There’s been an accident", "Emergency"),
    Example("Sorry, was a mistake", "False Alarm"),
    Example("I was scared but I'm fine", "False Alarm"),
    Example("False alarm, I don't need help", "False Alarm"),
    Example("No help needed around here", "False Alarm"),
    Example("Why does the device keep ringing if I'm fine?", "False Alarm"),
    Example("I'm fine, I'm fine, it was just a scare", "False Alarm"),
    Example("I don't need a doctor", "False Alarm"),
    Example("My heart is fine, I only have hippo", "False Alarm"),
    Example("I fell to the ground and despite having some pain, it's just a scratch", "False Alarm"),
    Example("You don't need to come", "False Alarm"),
    Example("I can breathe perfectly fine", "False Alarm"),
    Example("I do not need to go to a hospital", "False Alarm"),
    Example("Don't call an ambulance", "False Alarm"),
    Example("Sorry, I have touched something that is not...", "False Alarm"),
    Example("Oops I touched something I shouldn't", "False Alarm"),
    Example("Alarm of what? let me sleep in peace", "False Alarm"),
    Example("Help me turn off this device, I'm fine", "False Alarm"),
    Example("No need to send help", "False Alarm"),
    Example("Oops", "False Alarm"),
    Example("Why is the device on now? Nothing happens to me", "False Alarm"),
]

In [ ]:
inputs=[
  transcriptions
]

In [ ]:
response = co.classify(
  model='large',
  inputs=inputs,
  examples=voice_samples,
)

print(response)

[Classification<prediction: "False Alarm", confidence: 0.85354054>]
